# Instruction tuning with LoRA

1. Instruction tuning을 위한 데이터셋 만들기

2. LoRA로 모델에다 PEFT 하기

3. 훈련한 모델 허깅페이스 업로드

## 1. Instruction tuning을 위한 데이터셋 만들기

In [1]:
!pip -q install openai

In [11]:
import requests
from bs4 import BeautifulSoup
from google.colab import userdata
from openai import OpenAI
import csv
import os
import yaml
import re
import time
from tqdm import tqdm

In [3]:
champions = [
    "aatrox", "ahri", "akali", "akshan", "alistar", "amumu", "anivia", "annie", "aphelios", "ashe",
    "aurelionsol", "azir", "bard", "belveth", "blitzcrank", "brand", "braum", "caitlyn", "camille",
    "cassiopeia", "chogath", "corki", "darius", "diana", "drmundo", "draven", "ekko", "elise",
    "evelynn", "ezreal", "fiddlesticks", "fiora", "fizz", "galio", "gangplank", "garen", "gnar",
    "gragas", "graves", "gwen", "hecarim", "heimerdinger", "illaoi", "irelia", "ivern", "janna",
    "jarvaniv", "jax", "jayce", "jhin", "jinx", "kaisa", "kalista", "karma", "karthus", "kassadin",
    "katarina", "kayle", "kayn", "kennen", "khazix", "kindred", "kled", "kogmaw", "leblanc", "leesin",
    "leona", "lillia", "lissandra", "lucian", "lulu", "lux", "malphite", "malzahar", "maokai",
    "masteryi", "milio", "missfortune", "mordekaiser", "morgana", "naafiri", "nami", "nasus",
    "nautilus", "neeko", "nidalee", "nilah", "nocturne", "nunu", "olaf", "orianna", "ornn",
    "pantheon", "poppy", "pyke", "qiyana", "quinn", "rakan", "rammus", "reksai", "rell", "renataglasc",
    "renekton", "rengar", "riven", "rumble", "ryze", "samira", "sejuani", "senna", "seraphine", "sett",
    "shaco", "shen", "shyvana", "singed", "sion", "sivir", "skarner", "sona", "soraka", "swain",
    "sylas", "syndra", "tahmkench", "taliyah", "talon", "taric", "teemo", "thresh", "tristana",
    "trundle", "tryndamere", "twistedfate", "twitch", "udyr", "urgot", "varus", "vayne", "veigar",
    "velkoz", "vex", "vi", "viego", "viktor", "vladimir", "volibear", "warwick", "monkeyking", "xayah",
    "xerath", "xinzhao", "yasuo", "yone", "yorick", "yuumi", "zac", "zed", "ziggs", "zilean", "zoe", "zyra"
]
print(len(champions))

base_url = "https://universe.leagueoflegends.com/ko_KR/story/champion/"

def scrape_champion_data(champion):
    url = base_url + champion + "/"
    try:
        response = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(f"Error: An exception occurred while requesting data for '{champion}': {e}")
        return None, None

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        korean_name = soup.find('title').text.split('-')[0].strip()

        meta_description = soup.find('meta', {'name': 'description'})
        if meta_description:
            background_story = meta_description.get('content').replace('\n', ' ').strip()
        else:
            background_story = None
            print(f"Error: No background story found for champion '{champion}'.")

        return korean_name, background_story
    else:
        print(f"Error: Failed to fetch data for champion '{champion}'. HTTP status code: {response.status_code}")
        return None, None

with open("champion_bs.csv", "w", newline='', encoding='utf-8') as csvfile:
    fieldnames = ['url-name', 'korean-name', 'background-story']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for champion in tqdm(champions, desc="Scraping champions"):
        korean_name, background_story = scrape_champion_data(champion)
        if korean_name and background_story:
            writer.writerow({
                'url-name': champion,
                'korean-name': korean_name,
                'background-story': background_story
            })

162


Scraping champions: 100%|██████████| 162/162 [00:51<00:00,  3.15it/s]


In [4]:
client = OpenAI(
    api_key=userdata.get('OPENAI_TOKEN')
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o-mini",
)

chat_completion.choices[0].message.content

'This is a test. How can I assist you further?'

In [13]:
client = OpenAI(
    api_key=userdata.get('OPENAI_TOKEN')
)

base_prompt = """
위 글은 온라인 게임 리그 오브 레전드의 한 캐릭터의 배경 설명입니다.
이 캐릭터의 배경과 관련된 질문과 답변을 생성해 주세요.
다음 단계를 따라 주세요:

1. 생각하기 단계 (`<thinking>` 태그):
   - 생성할 질문과 답변에 대해 먼저 생각해 보세요.
   - 캐릭터의 핵심 정보, 중요한 사건, 행동 또는 결정과 그 결과에 관한 질문을 고려하세요.

2. 질문과 답변 작성 (`<qna>` 태그):
   - 질문은 짧고 명확하게 작성하세요.
   - 답변은 구체적이며 텍스트에서 직접적으로 추론할 수 있도록 작성하세요.
   - 아래 형식을 참고하여 질문과 답변을 작성하세요.

형식 예시:
<qna>
- q: "대부분의 필멸자가 알고 있는 현실 차원은 무엇인가?"
  a: "대부분의 필멸자는 물질 세계라는 하나의 현실 차원만 알고 있다."
- q: "오로라가 유년 시절을 보낸 곳은 어디인가?"
  a: "오로라는 브뤼니 부족의 고향이자 외딴 마을인 아무우에서 유년 시절을 보냈다."
- q: "오로라가 자신을 이해해준 유일한 가족 구성원은 누구인가?"
  a: "오로라의 이모할머니 하부우가 오로라를 진심으로 받아들였다."
- q: "오로라가 유년 시절에 보고 경험한 세계는 어떤 세계였나?"
  a: "오로라는 필멸 세계와 영혼 세계의 장막을 꿰뚫는 특별한 세계를 경험했다."
- q: "오로라는 무엇을 연구하며 기록했는가?"
  a: "오로라는 아무우의 영혼과 뒤얽힌 세계를 연구하며 세심하게 기록했다."
</qna>
"""

def generate_questions_and_answers(story):
    prompt = story + "\n\n" + base_prompt
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="gpt-4o-mini",
    )
    return chat_completion.choices[0].message.content.strip()

def extract_qna(content):
    match = re.search(r"<qna>(.*?)</qna>", content, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def parse_qna(qna_content):
    try:
        qa_list = yaml.safe_load(qna_content)
        if isinstance(qa_list, list) and all(isinstance(item, dict) for item in qa_list):
            qa_pairs = []
            for item in qa_list:
                if 'q' in item and 'a' in item:
                    qa_pairs.append((item['q'], item['a']))
                else:
                    raise ValueError("Each Q&A pair must contain 'q' and 'a' keys.")
            return qa_pairs
        else:
            raise ValueError("Q&A content is not a list of dictionaries.")
    except yaml.YAMLError as e:
        raise ValueError(f"YAML parsing error: {e}")

def write_qa_to_csv(qa_pairs):
    with open('champion_qa.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for question, answer in qa_pairs:
            writer.writerow([question, answer])

def generate_qa_from_bs(file_path):
    successful_champions = 0
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        champions = list(reader)
        total_champions = len(champions)
        for index, row in enumerate(champions, start=1):
            url_name = row['url-name']
            korean_name = row['korean-name']
            background_story = row['background-story']
            display_name = f"{korean_name} ({url_name})"
            print(f"Processing {display_name}...")
            attempts = 0
            while attempts < 3:
                try:
                    output = generate_questions_and_answers(background_story)
                    qna_content = extract_qna(output)
                    if not qna_content:
                        raise ValueError("No <qna> tags found in the response.")
                    qa_pairs = parse_qna(qna_content)
                    write_qa_to_csv(qa_pairs)
                    successful_champions += 1
                    print(f"Successfully wrote Q&A for {display_name} ({successful_champions}/{total_champions})")
                    break
                except Exception as e:
                    attempts += 1
                    print(f"Error processing {display_name}: {e}. Attempt {attempts}/3.")
                    if attempts < 3:
                        time.sleep(1)
                    else:
                        print(f"Failed to generate valid Q&A for {display_name} after 3 attempts.")
    print(f"\nTotal number of champions successfully processed: {successful_champions}")

with open('champion_qa.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["q", "a"])

generate_qa_from_bs('champion_bs.csv')

Processing 아트록스 (aatrox)...
Successfully wrote Q&A for 아트록스 (aatrox) (1/162)
Processing 아리 (ahri)...
Successfully wrote Q&A for 아리 (ahri) (2/162)
Processing 아칼리 (akali)...
Successfully wrote Q&A for 아칼리 (akali) (3/162)
Processing 아크샨 (akshan)...
Successfully wrote Q&A for 아크샨 (akshan) (4/162)
Processing 알리스타 (alistar)...
Successfully wrote Q&A for 알리스타 (alistar) (5/162)
Processing 아무무 (amumu)...
Successfully wrote Q&A for 아무무 (amumu) (6/162)
Processing 애니비아 (anivia)...
Successfully wrote Q&A for 애니비아 (anivia) (7/162)
Processing 애니 (annie)...
Successfully wrote Q&A for 애니 (annie) (8/162)
Processing 아펠리오스 (aphelios)...
Successfully wrote Q&A for 아펠리오스 (aphelios) (9/162)
Processing 애쉬 (ashe)...
Successfully wrote Q&A for 애쉬 (ashe) (10/162)
Processing 아우렐리온 솔 (aurelionsol)...
Successfully wrote Q&A for 아우렐리온 솔 (aurelionsol) (11/162)
Processing 아지르 (azir)...
Successfully wrote Q&A for 아지르 (azir) (12/162)
Processing 바드 (bard)...
Successfully wrote Q&A for 바드 (bard) (13/162)
Processing 벨베스 (b

In [14]:
from google.colab import files

files.download('champion_qa.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. LoRA로 모델에다 PEFT 하기

In [1]:
!pip -q install datasets peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import torch
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, setup_chat_format
import bitsandbytes as bnb
from google.colab import userdata
from huggingface_hub import login

login(userdata.get('HF_TOKEN'))

In [3]:
BASE_MODEL = "google/gemma-2-2b-it"
OUTPUT_MODEL_PATH = "./gemma-2-2b-it-lol"

In [4]:
df = pd.read_csv("champion_qa.csv")
df.head()

,q,a
0,"아트록스는 어떤 존재였으며, 처음에 어떤 역할을 했는가?","아트록스는 슈리마의 용맹한 전사로서 태양 원판의 화신이며, 고귀한 전투의 선두에서 ..."
1,아트록스가 다르킨으로 불리게 된 이유는 무엇인가?,"공허의 위협 이후, 아트록스와 그의 동료 초월체들이 타락하면서 필멸자들이 그들을 경..."
2,아트록스가 공허와의 전투에서 잃은 것은 무엇인가?,"아트록스와 그의 동족은 공허와의 전투에서 영원한 변화를 겪었으며, 고귀했던 본질과 ..."
3,아트록스는 어떻게 그의 검에 영원히 갇히게 되었는가?,타곤의 여명의 성위가 가르친 수단에 따라 필멸자들이 아트록스를 속이고 그의 힘을 검...
4,현재 아트록스의 목표는 무엇인가?,"아트록스는 자신의 절망과 고통을 끝내기 위해 완전한 파멸을 받아들이고, 이를 통해 ..."


In [5]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
dataset = Dataset.from_pandas(df)

def format_chat_prompt(record):
    chat_data = [
        {"role": "user", "content": record["q"]},
        {"role": "assistant", "content": record["a"]}
    ]
    record["qna"] = tokenizer.apply_chat_template(chat_data, tokenize=False)
    return record

dataset = dataset.map(format_chat_prompt, num_proc=4)
print(len(dataset))

Map (num_proc=4):   0%|          | 0/1239 [00:00<?, ? examples/s]

1239


In [7]:
dataset[523]

{'q': '루시안이 어릴 때부터 품었던 소망은 무엇인가?',
 'a': '루시안은 아버지 유리아스처럼 빛의 감시단에 들어가는 것이 소망이었다.',
 'qna': '<bos><start_of_turn>user\n루시안이 어릴 때부터 품었던 소망은 무엇인가?<end_of_turn>\n<start_of_turn>model\n루시안은 아버지 유리아스처럼 빛의 감시단에 들어가는 것이 소망이었다.<end_of_turn>\n'}

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
torch_dtype = torch.float16
attn_implementation = "eager"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quantization_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [21]:
def generate_response(prompt, model, tokenizer, temperature=0.1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=temperature > 0,
        temperature=temperature
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

test_prompt = """<start_of_turn>user
"다리우스에게 '녹서스의 실력자'라는 칭호를 준 사람은?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

test_prompt = """<start_of_turn>user
"이즈리얼 부모님의 어떤 직업은?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

test_prompt = """<start_of_turn>user
"신지드는 왜 자운으로 이주했나?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

<bos><start_of_turn>user
"다리우스에게 '녹서스의 실력자'라는 칭호를 준 사람은?<end_of_turn>
<start_of_turn>model
 다리우스에게 '녹서스의 실력자'라는 칭호를 준 사람은 **'녹서스'**입니다. 

다리우스는 녹서스의 실력을 인정하고, 그를 '녹서스의 실력자'라고 부르는 것을 칭찬하는 듯 합니다. 

이 칭호는 녹서스의 뛰어난 능력과 다리우스의 존경을 보여주는 듯 합니다. 
<end_of_turn>
<bos><start_of_turn>user
"이즈리얼 부모님의 어떤 직업은?<end_of_turn>
<start_of_turn>model
죄송하지만, 저는 이즈리얼 부모님의 직업에 대한 정보를 가지고 있지 않습니다. 

저는 인공지능 모델이기 때문에 개인 정보에 접근할 수 없으며, 특정 사람들의 정보를 제공할 수 없습니다. 

이즈리얼 부모님의 직업에 대해 더 자세히 알고 싶다면, 직접 이즈리얼에게 물어보는 것이 가장 좋습니다. 😊 
<end_of_turn>
<bos><start_of_turn>user
"신지드는 왜 자운으로 이주했나?<end_of_turn>
<start_of_turn>model
죄송하지만, 답변을 드릴 수 없습니다. 

"신지드는 왜 자운으로 이주했나?"라는 질문은 좀 더 구체적인 정보가 필요합니다. 

* **신지드**는 누구인가요? 
* **자운**은 어떤 의미인가요? 

질문에 대한 답변을 드릴 수 있도록 더 자세한 정보를 제공해주시면 감사하겠습니다. 
<end_of_turn>


In [10]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm)

In [11]:
total_layers = 0
for name, module in model.named_modules():
    # print(name, type(module))
    total_layers += 1
print(f"Total number of layers: {total_layers}")

Total number of layers: 422


In [12]:
total_layers = 0
for name, module in model.named_modules():
    if isinstance(module, bnb.nn.Linear4bit):
        names = name.split('.')
        layer_name = names[-1]
        if layer_name != 'lm_head':
            # print(name, type(module))
            total_layers += 1
print(f"Total number of linear layers: {total_layers}")

Total number of linear layers: 182


In [13]:
def find_linear_layers(model):
    linear_layers = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split('.')
            layer_name = names[-1]
            if layer_name != 'lm_head':
                linear_layers.add(layer_name)
    return list(linear_layers)

lora_target_modules = find_linear_layers(model)
lora_target_modules

['down_proj', 'o_proj', 'q_proj', 'gate_proj', 'v_proj', 'k_proj', 'up_proj']

In [14]:
lora_config = LoraConfig(
    r=16, # rank
    lora_alpha=32, # W + (alpha/rank) * (A@B)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=lora_target_modules
)

model = get_peft_model(model, lora_config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
        

In [15]:
lora_a_count = 0
lora_b_count = 0
for name, module in model.named_modules():
    if "lora" in name.lower():
        if isinstance(module, torch.nn.Linear):
            if "lora_A" in name:
                lora_a_count += 1
            elif "lora_B" in name:
                lora_b_count += 1
        # print(name, type(module))
print(f"LoRA A matrices: {lora_a_count}")
print(f"LoRA B matrices: {lora_b_count}")
print(f"Total LoRA rank pairs (A+B): {lora_a_count + lora_b_count}")

LoRA A matrices: 182
LoRA B matrices: 182
Total LoRA rank pairs (A+B): 364


In [16]:
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_PATH,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    logging_strategy="epoch",
    optim="paged_adamw_32bit",
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="no",
    do_eval=False,
    group_by_length=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="qna",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

trainer.train()

trainer.model.save_pretrained(OUTPUT_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

print(f"Training completed. Model saved to {OUTPUT_MODEL_PATH}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/1239 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
124,1.783500
248,1.250900
372,0.871400
496,0.556200
620,0.340100
744,0.236900
868,0.189800
992,0.165000
1116,0.147800
1240,0.138200


Training completed. Model saved to ./gemma-2-2b-it-lol


In [21]:
def generate_response(prompt, model, tokenizer, temperature=0.1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=temperature > 0,
        temperature=temperature
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

test_prompt = """<start_of_turn>user
"다리우스에게 '녹서스의 실력자'라는 칭호를 준 사람은?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

test_prompt = """<start_of_turn>user
"이즈리얼 부모님의 어떤 직업은?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

test_prompt = """<start_of_turn>user
"신지드는 왜 자운으로 이주했나?<end_of_turn>
<start_of_turn>model
"""
response = generate_response(test_prompt, model, tokenizer)
print(response)

<bos><start_of_turn>user
"다리우스에게 '녹서스의 실력자'라는 칭호를 준 사람은?<end_of_turn>
<start_of_turn>model
'녹서스의 실력자'라는 칭호는 스웨인이 직접 하사한 것이다.<end_of_turn>
<bos><start_of_turn>user
"이즈리얼 부모님의 어떤 직업은?<end_of_turn>
<start_of_turn>model
이즈리얼의 부모님은 뛰어난 연금술사였고, 치료 기술을 혁신하기 위해 노력했다.<end_of_turn>
<bos><start_of_turn>user
"신지드는 왜 자운으로 이주했나?<end_of_turn>
<start_of_turn>model
신지드는 필트오버에서 쫓겨난 후, 생명의 값어치가 크지 않은 자운에서 화학공학 관련 일을 찾기 위해 이주했다.<end_of_turn>


## 3. 훈련한 모델 허깅페이스 업로드

In [22]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import AutoPeftModelForCausalLM

In [23]:
NEW_MODEL_PATH = "./gemma-2-2b-it-lol"
MERGED_MODEL_PATH = "./gemma-2-2b-it-lol-merged"
BASE_MODEL = "google/gemma-2-2b-it"

In [34]:
model = AutoPeftModelForCausalLM.from_pretrained(
    NEW_MODEL_PATH,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map='cpu',
)

model = model.to('cpu')
merged_model = model.merge_and_unload()
merged_model.save_pretrained(MERGED_MODEL_PATH,
                             safe_serialization=True,
                             max_shard_size="2GB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
!du -sh */

2.8G	gemma-2-2b-it-lol/
4.9G	gemma-2-2b-it-lol-merged/
55M	sample_data/


In [37]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    MERGED_MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
HF_TOKEN=userdata.get('HF_WRITE_TOKEN')
REPO_NAME="gemma-2-2b-it-lol"

merged_model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=HF_TOKEN
)

tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=HF_TOKEN
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/star-bits/gemma-2-2b-it-lol/commit/eb58f80354a05101c05ddb8a3b573e2389bd1cfa', commit_message='Upload tokenizer', commit_description='', oid='eb58f80354a05101c05ddb8a3b573e2389bd1cfa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/star-bits/gemma-2-2b-it-lol', endpoint='https://huggingface.co', repo_type='model', repo_id='star-bits/gemma-2-2b-it-lol'), pr_revision=None, pr_num=None)

In [39]:
pipe = pipeline(
    "text-generation",
    model="star-bits/gemma-2-2b-it-lol",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "user",
     "content": "신지드는 왜 자운으로 이주했나?"},
]

outputs = pipe(messages, max_new_tokens=256)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
assistant_response

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

'신지드는 필트오버에서 쫓겨난 후, 생명의 값어치가 크지 않은 자운에서 화학공학 관련 일을 찾기 위해 이주했다.'